# Prognozowanie sprzedaży Walmartu za pomocą wybranych metod modelowania

Problemem jaki zdecydowaliśmy się opisać i przeanalizować jest przewidywanie sprzedaży sieci dyskontów w USA Walmart, a więc jest to problem regresji. Celem tego projektu jest zbudowanie rozwiązania, które będzie dobrze prognozowało sprzedaż Walmartu, a także wskazywało jakie czynniki mają największe znacznenie. Aby to osiągnąć przetestujemy różne techniki modelowania i wybierzemy najlepszą z nich, a potem zbudujemy na jej podstawie wyniki i prognozę sprzedaży. 

Zbiór danych wejściowych, który posłuży nam do analizy składa się z kilku podzbiorów:
* dane Walmart Sales Forecast pochodzące ze źródła Kaggle (https://www.kaggle.com/datasets/aslanahmedov/walmart-sales-forecast/data) - podstawowy zbiór zawierający informacje na temat sprzedaży, sklepów, a także elementów popytu naturalnego,
* dane Google Trends dotyczące słów powiązanych z badanym problemem,
* dane o wskaźnikach retail trade pochodzące ze strony US Census Bereau (https://www.census.gov/retail/marts/www/timeseries.html?utm_source=chatgpt.com),
* dane na temat świąt państwowych w USA pobrane za pomocą API Nager.Date
* wskaźniki Gross Domestic Product oraz M2 Money Supply pobrane ze strony FRED (https://fred.stlouisfed.org)

Finalnie nasz zbiór danych surowych został przygotowany za pomocą skryptu **data_downloading.ipynb** i składa się z następujących plików:
* train_data_final_df.csv - główny zbiór danych, który posłuży do trenowania i walidacji modeli,
* forecasts_data_final_df.csv - zbiór danych, który posłuży do testowania modeli i generowania prognoz.

Oba zbiory składają się z tej samej ilości takich samych zmiennych, natomiast różnią się jedynie zakresem dat. Zbiór do trenowania zaczyna się od 2010-02-07 i kończy na 2012-10-28, a zbiór do prognoz zaczyna się od 2012-11-04 i kończy na 2013-07-28. Warto również dodać, że oprócz daty mamy również informację o numerze sklepu, więc nasz problem dotyczy regresji, a dokładniej regresji panelowej, gdyż mamy dane rozłożone w czasie dla różnych jednostek. Poniżej znajduje się opis każdej zmiennej surowej, który pomoże w lepszym zrozumieniu zbioru oraz wyjaśnieniu niektórych wskaźników.